# Common Built‑in Exceptions
- Python ships with a rich hierarchy of **exception classes**; most automation errors fall into a small, predictable subset.  
- All ordinary **run‑time exceptions** inherit from `Exception`, but subclasses convey *why* something failed (e.g., file missing vs. wrong type).  
- Catching overly broad bases like `Exception` **hides root causes** and can **mask bugs—prefer** the narrowest class you can handle.  
- Understanding the inheritance tree lets you decide when a single `except` can cover many related problems (e.g., `OSError`).  

In [56]:
import inspect, builtins

def show_tree(base, level=0, max_depth=1):
    # Base case
    if level > max_depth:
        return

    for name, obj in vars(builtins).items():
        if inspect.isclass(obj) and issubclass(obj, base) and obj is not base:
            print("\t" * level + f"- {name}")
            show_tree(obj, level + 1, max_depth)

show_tree(Exception, max_depth=1)

- ArithmeticError
	- FloatingPointError
	- OverflowError
	- ZeroDivisionError
- AssertionError
- AttributeError
- BufferError
- EOFError
- ImportError
	- ModuleNotFoundError
- LookupError
	- IndexError
	- KeyError
- MemoryError
- NameError
	- UnboundLocalError
- OSError
	- BlockingIOError
	- ChildProcessError
	- ConnectionError
	- FileExistsError
	- FileNotFoundError
	- InterruptedError
	- IsADirectoryError
	- NotADirectoryError
	- PermissionError
	- ProcessLookupError
	- TimeoutError
	- BrokenPipeError
	- ConnectionAbortedError
	- ConnectionRefusedError
	- ConnectionResetError
- ReferenceError
- RuntimeError
	- NotImplementedError
	- RecursionError
- StopAsyncIteration
- StopIteration
- SyntaxError
	- IndentationError
	- TabError
- SystemError
- TypeError
- ValueError
	- UnicodeError
	- UnicodeDecodeError
	- UnicodeEncodeError
	- UnicodeTranslateError
- Warning
	- BytesWarning
	- DeprecationWarning
	- EncodingWarning
	- FutureWarning
	- ImportWarning
	- PendingDeprecationWarning
	- Re

## `OSError` Family: Filesystem & Network Issues
- Signals problems interacting with the **operating system**: files, permissions, paths, sockets.  
- Subclasses such as `FileNotFoundError`, `PermissionError`, `IsADirectoryError`, `ConnectionRefusedError`, and `TimeoutError` offer granularity.  
- Catch individual subclasses when you can recover differently (create a missing file, prompt for sudo, retry a connection).  
- A single `except OSError` still groups all OS‑level failures when the response is the same (e.g., log and abort).  

In [7]:
try:
    with open('nonexistent.txt', 'r') as file:
        content = file.read()
except FileNotFoundError as e:                           # try to catch more specific error
    print(f"File not found: {e}")
except PermissionError:                                  # try to catch more speicific error
    print("Permission denied when accessing resources.")
except OSError as os_err:                                # catch all OSError
    print(f"General OS error: {os_err}")

File not found: [Errno 2] No such file or directory: 'nonexistent.txt'


## `KeyError`: Missing Dictionary Keys
- Raised when using `dict[key]` with a key that is absent.  
- Frequent in config loading, JSON parsing, or environment variable maps.  
- Mitigation patterns: `dict.get(key, default)`, membership tests (`if key in cfg`), or a tailored `except KeyError`.  
- Treats missing data distinctly from a wrong value (`ValueError`) or wrong type (`TypeError`).  

In [5]:
config = {"host": "server1", "port": 8080}
config2 = {"host": "server1", "port": 8080, "api-key": "12345"}

api_key = config.get("api-key", "") # use config.get() to return default value if key is absent


# Using conditionals (Look Before You Leap approach)
def call_endpoint(config, endpoint):
    """Calls the specified endpoint of the configured host.

    Args:
        config (dict(str)): Dict containing host, port, and api-key
        endpoint (str): The endpoint to hit
    """
    if "api-key" in config:
        print(f"Making API call to endpoint {endpoint} with key: {config["api-key"]}")
    else:
        print("No api-key available, not possible to call API.")


# Using the exception (Easier to Ask for Forgiveness than Permission approach)
def call_endpoint_exception(config, endpoint):
    """Calls the specified endpoint of the configured host.

    Args:
        config (dict(str)): Dict containing host, port, and api-key
        endpoint (str): The endpoint to hit
    """
    try:
        print(f"Making API call to endpoint {endpoint} with key: {config["api-key"]}")
    except KeyError as missing_key:
        print(f"No required key {missing_key} available, not possible to call API.")
    

# call_endpoint(config, "/users")
# call_endpoint(config2, "/users")

call_endpoint_exception(config, "/users")
call_endpoint_exception(config2, "/users")

No required key 'api-key' available, not possible to call API.
Making API call to endpoint /users with key: 12345


## `IndexError`: Sequence Index Out of Bounds
- Triggered when list/tuple indices fall outside the valid range: **negative beyond the left edge** or **≥** `len(seq)`.  
- Common during iterative processing of dynamic lists or user‑provided indexes.  
- Prevent with **bounds checks** (`if i < len(seq)`), **safe iteration** (`for item in seq:`), or **catch and default**.  
- Signals "wrong position" rather than "wrong content".  

In [44]:
servers = ["web01", "web02"]
i = 2

# LBYL approach
if i < len(servers):
    print(servers[i])

# EAFP approach
try:
    print(servers[i])
except IndexError as e:
    print(f"Index error: {e}. List length is {len(servers)}")
    

Index error: list index out of range. List length is 2


## `ValueError` vs. `TypeError`
- **ValueError**: argument type is acceptable but content/value is invalid (e.g., `int("abc")`).  
- **TypeError**: operation applied to an object of the wrong type altogether (e.g., `len(5)` or `"a" + 3`).  
- Distinguishing them clarifies whether to validate *content* or convert *types*.  
- Catch them separately to craft precise user feedback.  

In [43]:
try:
    #port = int("8080") # OK
    port = int("http")  # ValueError, since literal must be a valid base-10 value
except ValueError as e:
    print(f"Bad numeric string: {e}")

try:
    total = "Errors: " + 5
except TypeError as e:
    print(f"Type mismatch: {e}")
    

Bad numeric string: invalid literal for int() with base 10: 'http'
Type mismatch: can only concatenate str (not "int") to str


## `AttributeError`: Missing Object Member
- Raised when an attribute or method doesn't exist on the object referenced.  
- Often results from typos, unexpected `None`, or polymorphic functions returning different types.  
- Defensive techniques: `hasattr(obj, "attr")`, `if obj is not None:`, or narrow `except AttributeError`.  
- Conveys "object of this type doesn’t support that capability".  

In [46]:
class Calculator:
    def add(self, a, b):
        return a + b

calc = Calculator()

# LBYL approach
if hasattr(calc, "subtract"):
    print(calc.subtract(10, 5))
else:
    print("Object has no attribute 'subtract'")


# EAFP approach
try:
    print(calc.subtract(10, 5))
except AttributeError as e:
    print(f"AttributeError caught: {e}.")

try:
    print(calc.result)
except AttributeError as e:
    print(f"AttributeError caught: {e}.")

Object has no attribute 'subtract'
AttributeError caught: 'Calculator' object has no attribute 'subtract'.
AttributeError caught: 'Calculator' object has no attribute 'result'.


## `ImportError` / `ModuleNotFoundError`
- Raised when an `import` statement cannot locate a module/package.  
- `ModuleNotFoundError` (Python 3.6+) is the specific subclass; catching `ImportError` also covers it.  
- Causes: misspelling, package not installed, wrong virtual environment, or PYTHONPATH issues.  
- Typical handling logs instructions and aborts early to avoid cascading failures.  

In [52]:
try:
    import non_existent_lib
except ModuleNotFoundError as e:
    print(f"Import failed: {e}. Is the library installed and the correct venv active?")
    

Import failed: No module named 'non_existent_lib'. Is the library installed and the correct venv active?


## Hands-on Exercise

In [1]:
def check_service_status(port):
    print("Initiating check...")
    log = []
    try:
        if port == 80:
            log.append("OK")
        elif port == 503:
            raise ConnectionRefusedError("Service unavailable")
        else:
            log.append("Unknown")
    except ConnectionRefusedError:
        log.append("FAIL")
        print("Connection failed.")
    # Executes if `try` block executes successfully
    else:
        log.append("SUCCESS")
        print("Check completed without errors.")
    # Executes regardless of exceptions were raised or not
    finally:
        log.append("LOGGED")
        print("Finalizing check.")
    return log

status = check_service_status(80)
print(f"Status: {status}")

Initiating check...
Check completed without errors.
Finalizing check.
Status: ['OK', 'SUCCESS', 'LOGGED']


In [2]:
def summarize_logs(log_entries):
    summary = []
    for entry in log_entries:
        try:  
            user = entry.get('user', 'system')    # 'user' key is optional
            event_id = entry['event_id']          # 'event_id' key is mandatory
            summary.append(f"{event_id}: {user}")
        except KeyError:
            summary.append("ERROR: Missing-Data")
        except (TypeError, AttributeError):
            summary.append("ERROR: Invalid-Entry")
 
    return summary
 
logs = [
    {'event_id': 101, 'user': 'alice'},
    {'event_id': 102},
    None,               # will raise TypeError as the code will attempt entry.get() 
    {'user': 'bob'}     # will raise KeyError as the code will attempt entry['event_id']
]
 
print(summarize_logs(logs))

['101: alice', '102: system', 'ERROR: Invalid-Entry', 'ERROR: Missing-Data']


In [69]:
# Inefficient version
def get_metric(data, server_id, metric_name):
    try:
        value = data[server_id][metric_name]
        return f"Metric '{metric_name}' on server '{server_id}' is {value}"
    except:
        return f"Could not retrieve metric '{metric_name}' for server '{server_id}'."


server_data = {
    "srv-web-01": {"cpu": 0.75, "memory": 0.5},
    "srv-db-01": {"cpu": 0.4}
}
# print(get_metric(server_data, "srv-db-01", "memory"))

# Appropriate version
# Only excepting BaseException catches everhthing including SystemExit, KeyboardInterrupt and even SyntaxError.
# Moreover, it will silently hide unrelated bugs such as TypeError if data was accidently passed as None.
# The handler should be specific like KeyError, TypeError to only catch the expected errors.
def get_metric_improved(data, server_id, metric_name):
    try:
        value = data[server_id][metric_name]
        return f"Metric '{metric_name}' on server '{server_id}' is {value}"
    except KeyError as key_missing:
        return f"Could not retrieve metric '{metric_name}' for server '{server_id}' with error: {key_missing} missing."
    except TypeError as err:
        return f"Could not retrieve metric '{metric_name}' for server '{server_id}' with error: {err}."

print(get_metric_improved(server_data, "srv-db-01", "memory"))


Could not retrieve metric 'memory' for server 'srv-db-01' with error: 'memory' missing.


## Questions
#### Question 1
A script is being designed to process a list of hostnames. For each hostname, it will connect, download a configuration file, and parse it. The network is known to be unreliable, and some configuration files might be malformed or missing expected keys.

Which of the following best describes the most Pythonic and robust design philosophy for this script?

1. The **LYBL (Look Before You Leap)** approach where the script  first pings each host, then send a request to check if the file exists, then checks file permissions and only then attempts to download and parse it.
2. The **EAFP (Easier to Ask for Forgiveness than Permission)** approach, where the script directly attempt to connect, download and parse the file inside a `try` block using specific `except` block to handle `ConnectionError`, `FileNotFoundError` or `KeyError` as they occur.

**Answer:** Option 2 is preferred Pythonic approach for this scenario. It results in cleaner, more readable code by focusing the `try` block on the primary task (**happy path**). It robustly handles specific, expected errors in separate except blocks, clearly separating the main logic from error-handling logic.



#### Question 2
Which of the following is the most significant advantage of defining a custom exception hierarchy such as a base `ServiceError` with subclasses `AutheticationError` 
and `QuotaExceedError`?

1. It automatically generates more detailed traceback messages than built-in exceptions.
2. It allows the calling code to use a single `except ServiceError:` block to handle all service-related failures or to use specific `except AuthenticationError` blocks for targeted recovery logic.

**Answer:** Option 2 because Callers can  handle errors at different levels of granularity such as:
- catch specific subclass to retry/refresh token.
- catch base class to perform generic logging and cleanup for any related failure.
